In [1]:
import numpy as np
import pandas as pd

In [2]:
x=pd.read_csv

In [3]:
data=pd.read_csv("PlayTennis.csv")

In [4]:
data.head()

Outlook Temperature Humidity    Wind Play Tennis
0     Sunny         Hot     High    Weak          No
1     Sunny         Hot     High  Strong          No
2  Overcast         Hot     High    Weak         Yes
3      Rain        Mild     High    Weak         Yes
4      Rain        Cool   Normal    Weak         Yes

In [5]:
data.describe()

Outlook Temperature Humidity  Wind Play Tennis
count       14          14       14    14          14
unique       3           3        2     2           2
top      Sunny        Mild     High  Weak         Yes
freq         5           6        7     8           9

In [6]:
data[data["Outlook"]=="Rain"]

Outlook Temperature Humidity    Wind Play Tennis
3     Rain        Mild     High    Weak         Yes
4     Rain        Cool   Normal    Weak         Yes
5     Rain        Cool   Normal  Strong          No
9     Rain        Mild   Normal    Weak         Yes
13    Rain        Mild     High  Strong          No

In [7]:
# total entrophy of all columns

x={}
def Total_Entrophy(data):
    columns=data.columns
    for c in columns:
        entr=0
        x1 = data[c].unique()
        for j in x1:
            rows = data[data[c]==j].shape[0]
            row_total = data.shape[0]
            entr=entr + -1*(rows/row_total)*(np.log2(rows/row_total))
        x[c]=entr
Total_Entrophy(data)        


In [8]:
data.describe()



Outlook Temperature Humidity  Wind Play Tennis
count       14          14       14    14          14
unique       3           3        2     2           2
top      Sunny        Mild     High  Weak         Yes
freq         5           6        7     8           9

In [9]:
x    # total entropy

{'Outlook': 1.5774062828523454,
 'Temperature': 1.5566567074628228,
 'Humidity': 1.0,
 'Wind': 0.9852281360342515,
 'Play Tennis': 0.9402859586706311}

In [10]:
# E(feature|"Play Tennis")

x_H={}
def H(data):
    columns=data.columns
    for c in columns:
        entr1=0
        x1 = data[c].unique()
        for j in x1:
            rows1 = data[data[c]==j][data["Play Tennis"]=="Yes"].shape[0]
            rows2 = data[data[c]==j][data["Play Tennis"]=="No"].shape[0]
            row=data[data[c]==j].shape[0]
            row_total = data.shape[0]
            # entr1 = entr1 + -1*((rows1/row)*np.log2(rows1/row) +  (rows2/row)*np.log2(rows2/row))*(row/row_total)
            
            if(np.isnan(-1*((rows1/row)*np.log2(rows1/row) +  (rows2/row)*np.log2(rows2/row)))):
                continue
            else:    
                entr1=entr1+-1*((rows1/row)*np.log2(rows1/row) +  (rows2/row)*np.log2(rows2/row))*(row/row_total)
        x_H[c]=entr1        
                
            
H(data)        

C:\Users\archi\AppData\Local\Temp\ipykernel_19628\1010697907.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rows1 = data[data[c]==j][data["Play Tennis"]=="Yes"].shape[0]
C:\Users\archi\AppData\Local\Temp\ipykernel_19628\1010697907.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rows2 = data[data[c]==j][data["Play Tennis"]=="No"].shape[0]
C:\Users\archi\AppData\Local\Temp\ipykernel_19628\1010697907.py:16: RuntimeWarning: divide by zero encountered in log2
  if(np.isnan(-1*((rows1/row)*np.log2(rows1/row) +  (rows2/row)*np.log2(rows2/row)))):
C:\Users\archi\AppData\Local\Temp\ipykernel_19628\1010697907.py:16: RuntimeWarning: invalid value encountered in double_scalars
  if(np.isnan(-1*((rows1/row)*np.log2(rows1/row) +  (rows2/row)*np.log2(rows2/row)))):


In [11]:
x_H

{'Outlook': 0.6935361388961918,
 'Temperature': 0.9110633930116763,
 'Humidity': 0.7884504573082896,
 'Wind': 0.8921589282623617,
 'Play Tennis': 0}

In [12]:

info={}  # storing info gain of all columns wrt "Play Tennis" columns
def info_gain(data):
    column=data.columns
    for c in column:
        info[c]=x["Play Tennis"] - x_H[c]
        
info_gain(data)     


In [13]:
info

{'Outlook': 0.24674981977443933,
 'Temperature': 0.02922256565895487,
 'Humidity': 0.15183550136234159,
 'Wind': 0.04812703040826949,
 'Play Tennis': 0.9402859586706311}

In [14]:
# making subTree

def subTree(data,s):
    x=data[s].value_counts(sort=False)
    
    tree = {}
    for a,b in x.iteritems():
        
        Flag=False
        for c in ["Yes","No"]:
            count = data[data[s]==a][data["Play Tennis"]==c].shape[0]
            if(count == b):
                Flag=True
                tree[a]=c
                data = data[data[s] != a]
        if(Flag != True):
            tree[a]='?' 
            
    return tree , data

In [15]:
# generating Tree
#tree1={}

def Tree(root,Prev_root,data):
    if (data.shape[0]!=0):
        
        
        Total_Entrophy(data)
        H(data)
        info_gain(data)
        
        
        s='' # feature with maximum info gain
        m=0
        for i in info.keys():
            if(info[i] > m and i!='Play Tennis'):
                m=info[i]
                s=i
        
        
        tree1,data = subTree(data,s)
        next_root = None
        if(Prev_root != None):
            root[Prev_root] = dict()
            root[Prev_root][s] = tree1
            next_root = root[Prev_root][s]
        else:
            root[s] = tree1
            next_root = root[s]
            
        print(list(next_root.items()))    

        for node,branch in list(next_root.items()):
            
            if branch == '?':
                p = data[data[s]==node]
                Tree(next_root,node,p)
            
        
                


In [16]:
root2={}
Tree(root2,None,data)

[('Sunny', '?'), ('Overcast', 'Yes'), ('Rain', '?')]
[('High', 'No'), ('Normal', 'Yes')]
[('Weak', 'Yes'), ('Strong', 'No')]


C:\Users\archi\AppData\Local\Temp\ipykernel_19628\1010697907.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rows1 = data[data[c]==j][data["Play Tennis"]=="Yes"].shape[0]
C:\Users\archi\AppData\Local\Temp\ipykernel_19628\1010697907.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rows2 = data[data[c]==j][data["Play Tennis"]=="No"].shape[0]
C:\Users\archi\AppData\Local\Temp\ipykernel_19628\1010697907.py:16: RuntimeWarning: divide by zero encountered in log2
  if(np.isnan(-1*((rows1/row)*np.log2(rows1/row) +  (rows2/row)*np.log2(rows2/row)))):
C:\Users\archi\AppData\Local\Temp\ipykernel_19628\1010697907.py:16: RuntimeWarning: invalid value encountered in double_scalars
  if(np.isnan(-1*((rows1/row)*np.log2(rows1/row) +  (rows2/row)*np.log2(rows2/row)))):


In [17]:
root2    # FINAL DECISION TREE 

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}